In [ ]:
# 🏎️ F1 Race Outcome & Strategy Analyzer
# Beginner ML Project
# Author: Shubhangi Bhagat


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Plot settings
plt.style.use("default")


In [ ]:
drivers = pd.read_csv("../data/drivers.csv")
constructors = pd.read_csv("../data/constructors.csv")
driver_grid = pd.read_csv("../data/driverGrid.csv")
circuits = pd.read_csv("../data/circuits.csv")


In [ ]:
drivers.head()


In [ ]:
constructors.head()


In [ ]:
driver_grid.head()


In [ ]:
circuits.head()


In [ ]:
drivers.info()
constructors.info()
driver_grid.info()
circuits.info()


In [ ]:
# Select important columns from driver results
results_df = drivers[[
    'raceId',
    'start_position',
    'positionOrder',
    'points',
    'driver_forename',
    'driver_surname',
    'status'
]].copy()

results_df.head()


In [ ]:
results_df['driverName'] = (
    results_df['driver_forename'] + " " + results_df['driver_surname']
)

results_df.drop(['driver_forename', 'driver_surname'], axis=1, inplace=True)

results_df.head()


In [ ]:
results_df = results_df[results_df['positionOrder'] > 0]
results_df.reset_index(drop=True, inplace=True)

results_df.info()


In [ ]:
results_df = results_df[results_df['start_position'] > 0]
results_df.reset_index(drop=True, inplace=True)


In [ ]:
results_df.drop(columns=['status'], inplace=True)
results_df.head()


In [ ]:
plt.figure(figsize=(6,4))
plt.scatter(
    results_df['start_position'],
    results_df['positionOrder'],
    alpha=0.3
)

plt.xlabel("Starting Grid Position")
plt.ylabel("Finishing Position")
plt.title("Grid Position vs Finishing Position (Cleaned Data)")
plt.show()


In [ ]:
driver_avg_finish = (
    results_df
    .groupby('driverName')['positionOrder']
    .mean()
    .reset_index()
    .rename(columns={'positionOrder': 'driver_avg_finish'})
)

driver_avg_finish.head()


In [ ]:
results_df = pd.merge(
    results_df,
    driver_avg_finish,
    on='driverName',
    how='left'
)

results_df.head()


In [ ]:
results_df['grid_normalized'] = (
    results_df['start_position'] / results_df['start_position'].max()
)

results_df[['start_position', 'grid_normalized']].head()


In [ ]:
race_difficulty = (
    results_df
    .groupby('raceId')['positionOrder']
    .mean()
    .reset_index()
    .rename(columns={'positionOrder': 'race_difficulty'})
)

results_df = pd.merge(
    results_df,
    race_difficulty,
    on='raceId',
    how='left'
)

results_df.head()


In [ ]:
# Features (inputs)
X = results_df[
    [
        'start_position',
        'grid_normalized',
        'driver_avg_finish',
        'race_difficulty'
    ]
]

# Target (output)
y = results_df['positionOrder']

X.head(), y.head()


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42
)

X_train.shape, X_test.shape


In [ ]:
from sklearn.linear_model import LinearRegression

lr_model = LinearRegression()
lr_model.fit(X_train, y_train)


In [ ]:
y_pred = lr_model.predict(X_test)


In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

mae, rmse, r2


In [ ]:
plt.figure(figsize=(6,4))
plt.scatter(y_test, y_pred, alpha=0.3)
plt.xlabel("Actual Finishing Position")
plt.ylabel("Predicted Finishing Position")
plt.title("Actual vs Predicted Finishing Position")
plt.show()



In [ ]:
from sklearn.ensemble import RandomForestRegressor

rf_model = RandomForestRegressor(
    n_estimators=100,
    random_state=42,
    n_jobs=-1
)

rf_model.fit(X_train, y_train)


In [ ]:
rf_pred = rf_model.predict(X_test)

rf_mae = mean_absolute_error(y_test, rf_pred)
rf_rmse = np.sqrt(mean_squared_error(y_test, rf_pred))
rf_r2 = r2_score(y_test, rf_pred)

rf_mae, rf_rmse, rf_r2


In [ ]:
comparison = pd.DataFrame({
    'Model': ['Linear Regression', 'Random Forest'],
    'MAE': [mae, rf_mae],
    'RMSE': [rmse, rf_rmse],
    'R2': [r2, rf_r2]
})

comparison


In [ ]:
importances = rf_model.feature_importances_

feature_importance = pd.DataFrame({
    'Feature': X.columns,
    'Importance': importances
}).sort_values(by='Importance', ascending=False)

feature_importance


##  Key Insights from Feature Importance

- **Driver historical performance** is the strongest predictor of race outcome, contributing nearly 46% to the model’s decisions.
- **Starting grid position**, while important, contributes less than driver consistency, indicating that race dynamics allow skilled drivers to recover positions.
- **Race-level context** also plays a significant role, showing that not all races are equally predictable.
- These results validate the need for machine learning over simple rule-based predictions in Formula 1 analytics.


### Feature Importance Insights
The model relies most on historical driver performance, followed by starting grid position and race difficulty.
This aligns with real-world F1 dynamics, validating model behavior.


In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
print(f"R² Score: {r2:.3f}")


In [ ]:
import matplotlib.pyplot as plt

residuals = y_test - y_pred

plt.figure(figsize=(8, 5))
plt.scatter(y_pred, residuals, alpha=0.4)
plt.axhline(0, linestyle='--')
plt.xlabel("Predicted Finishing Position")
plt.ylabel("Residual (Actual - Predicted)")
plt.title("Residuals vs Predicted Finishing Position")
plt.show()


### Model Evaluation Summary

The model achieves reasonable accuracy in predicting F1 finishing positions.
Residual analysis shows mostly random distribution, indicating no strong bias.
Some higher finishing positions show larger variance, suggesting race unpredictability.


In [ ]:
plt.figure(figsize=(7, 4))
plt.hist(residuals, bins=30)
plt.xlabel("Residual")
plt.ylabel("Frequency")
plt.title("Distribution of Prediction Errors")
plt.show()


### Error Distribution Analysis

The prediction errors are approximately centered around zero, indicating no major systemic bias.
Most predictions fall within ±5 positions, which is reasonable for a highly stochastic sport like Formula 1.
The presence of long tails reflects race incidents and unpredictable events such as DNFs and safety cars.


In [ ]:
import os

os.makedirs("models", exist_ok=True)


In [ ]:
import os
import pickle

os.makedirs("models", exist_ok=True)

with open("models/f1_finish_position_model.pkl", "wb") as f:
    pickle.dump(rf_model, f)

print("Random Forest model saved successfully!")
